In [12]:
import numpy as np 
import matplotlib.pyplot as plt 


We generate a excitation signal $x(t)$, on the form

$$
x(t) = A_e \sin(\omega t)
$$

And sense a resulting signal $y(t)$

$$
y(t) = A_s \sin(\omega t + \phi)
$$

where only $A_s$ can change, note that it can take negative values. This value corelates to the position of the probe.

Rewriting $y(t)$ in order to solve as least square problem:

$$
y(t) = A_s (\sin(\omega t) \cos(\phi) + \cos(\omega t) \sin(\phi))
$$
$$
y(t) = A_s \cos(\phi) \sin(\omega t)  + A_s \sin(\phi) \cos(\omega t)
$$
$$
y(t) = a_0 \sin(\omega t)  + a_1 \cos(\omega t)
$$

where the original parameters can be found by
$$
\hat A_s = \sqrt{a_0^2 + a_1 ^2}
$$

$$
\hat \phi = \texttt{atan2}(a_1,a_0)
$$

This does pose one problem, $A_s$ is always positive, and $\phi$ adjusts in order to acomodate the sign change of $A_s$

In [13]:
from numpy import linalg


def ls_sine_ident(t,y):

    M = np.c_[np.sin(t), np.cos(t)]
    sol = linalg.lstsq(M,y,rcond=None)

    a0,a1 = sol[0]

    A = np.sqrt(a0**2 + a1**2)
    phi = np.atan2(a1,a0)
    phi = np.rad2deg(phi)
    phi = np.round(phi,2)

    return A,phi


In [14]:
t = np.linspace(0,2*np.pi)

phi_off = np.deg2rad(39)

y0 = np.sin(t + phi_off)
y1 = -np.sin(t + phi_off)

print(ls_sine_ident(t,y0))
print(ls_sine_ident(t,y1))


(np.float64(1.0000000000000002), np.float64(39.0))
(np.float64(1.0000000000000002), np.float64(-141.0))


One solution is to calibrate the *constant* phase offset $\phi$ and use it to compute the *sign true* $A_s^*$ as

$$
A_s^* = \hat A_s \cos(\hat \phi - \phi_c)
$$

where $\phi_c$ is the calibrated phase where $A_s$ is positive.

An even simpler alternative is to check if the angle deviates more that 180 deg, i.e.

This is pseudo-c, as I don't know all math function-calls in c
```c
float calculateAs(float a0, float a1){

    float As = Math.sqrt(a0*a0, a1*a1);
    float phi = Math.atan2(a1,a0);
    float deltaPhi = phi - phiCalib;

    if(Math.abs(deltaPhi)>Math.deg2rad(90)){
        As = -1.0*As;
    }
    return As;
}
```

The final least squares problem will be to identify the amplitude, phase and offset. Given $N$ measurements we need to solve


$$
M \theta = Y
$$


$$
M = 
\begin{bmatrix}
0 & 1 & 1 \\
\sin(\Omega) & \cos(\Omega) & 1 \\
\sin(2\Omega) & \cos(2\Omega) & 1 \\
\sin(3\Omega) & \cos(3\Omega) & 1 \\
\vdots & \vdots & \vdots \\
\sin((N-1)\Omega) & \cos((N-1)\Omega) & 1 \\
\end{bmatrix}
$$

$$
\begin{bmatrix}
y_0 \\
y_1 \\
y_2 \\
y_3 \\
\vdots \\
n_{N-1}
\end{bmatrix}
$$

One easy solution is to use the pseudo inverse by multipling each side with $M^T$

$$
M^T M \theta = M^T Y
$$

$$
\theta = (M^T M)^{-1} M^T Y
$$


$$
M = 
\begin{bmatrix}
m_{sin} & m_{cos} & m_1
\end{bmatrix}
$$
$$
M^T M = 
\begin{bmatrix}
m_{sin}^T m_{sin} & m_{sin}^T m_{cos} & m_{sin}^T m_1 \\
m_{cos}^T m_{sin} & m_{cos}^T m_{cos} & m_{cos}^T m_1 \\
m_1^T m_{sin} & m_1^T m_{cos} & m_1^T m_1 \\
\end{bmatrix}
$$

In [15]:

theta_true = np.array([0.1337, 0.42, 4.337])

M = np.c_[np.sin(t), np.cos(t), np.ones(len(t))]

y3 = M@theta_true 

#plt.plot(t,y3)

theta_hat = np.linalg.inv(M.T@M)@M.T@y3

print("True theta", theta_true)
print("Est theta", theta_hat)

m11 = 0 
m12 = 0
m13 = 0

m22 = 0
m23 = 0 
m33 = len(t)

MtY1 = 0 
MtY2 = 0 
MtY3 = 0 

Y = y3

for i,ti in enumerate(t):

    m11 += np.sin(ti)**2
    m12 += np.sin(ti)*np.cos(ti)
    m13 += np.sin(ti)

    m22 += np.cos(ti)**2
    m23 += np.cos(ti)

    MtY1 += np.sin(ti)*Y[i]
    MtY2 += np.cos(ti)*Y[i]
    MtY3 += Y[i]



True theta [0.1337 0.42   4.337 ]
Est theta [0.1337 0.42   4.337 ]


C code for matrix ionverse (from stack overflow)

```c
// computes the inverse of a matrix m
double det = m(0, 0) * (m(1, 1) * m(2, 2) - m(2, 1) * m(1, 2)) -
             m(0, 1) * (m(1, 0) * m(2, 2) - m(1, 2) * m(2, 0)) +
             m(0, 2) * (m(1, 0) * m(2, 1) - m(1, 1) * m(2, 0));

double invdet = 1 / det;

Matrix33d minv; // inverse of matrix m
minv(0, 0) = (m(1, 1) * m(2, 2) - m(2, 1) * m(1, 2)) * invdet;
minv(0, 1) = (m(0, 2) * m(2, 1) - m(0, 1) * m(2, 2)) * invdet;
minv(0, 2) = (m(0, 1) * m(1, 2) - m(0, 2) * m(1, 1)) * invdet;
minv(1, 0) = (m(1, 2) * m(2, 0) - m(1, 0) * m(2, 2)) * invdet;
minv(1, 1) = (m(0, 0) * m(2, 2) - m(0, 2) * m(2, 0)) * invdet;
minv(1, 2) = (m(1, 0) * m(0, 2) - m(0, 0) * m(1, 2)) * invdet;
minv(2, 0) = (m(1, 0) * m(2, 1) - m(2, 0) * m(1, 1)) * invdet;
minv(2, 1) = (m(2, 0) * m(0, 1) - m(0, 0) * m(2, 1)) * invdet;
minv(2, 2) = (m(0, 0) * m(1, 1) - m(1, 0) * m(0, 1)) * invdet;

```

In [ ]:


def inv_m(m):

    minv = np.zeros((3,3))
    det = m[0, 0] * (m[1, 1] * m[2, 2] - m[2, 1] * m[1, 2]) -  m[0, 1] * (m[1, 0] * m[2, 2] - m[1, 2] * m[2, 0]) +  m[0, 2] * (m[1, 0] * m[2, 1] - m[1, 1] * m[2, 0])

    invdet = 1 / det

    minv[0, 0] = (m[1, 1] * m[2, 2] - m[2, 1] * m[1, 2]) * invdet
    minv[0, 1] = (m[0, 2] * m[2, 1] - m[0, 1] * m[2, 2]) * invdet
    minv[0, 2] = (m[0, 1] * m[1, 2] - m[0, 2] * m[1, 1]) * invdet
    minv[1, 0] = (m[1, 2] * m[2, 0] - m[1, 0] * m[2, 2]) * invdet
    minv[1, 1] = (m[0, 0] * m[2, 2] - m[0, 2] * m[2, 0]) * invdet
    minv[1, 2] = (m[1, 0] * m[0, 2] - m[0, 0] * m[1, 2]) * invdet
    minv[2, 0] = (m[1, 0] * m[2, 1] - m[2, 0] * m[1, 1]) * invdet
    minv[2, 1] = (m[2, 0] * m[0, 1] - m[0, 0] * m[2, 1]) * invdet
    minv[2, 2] = (m[0, 0] * m[1, 1] - m[1, 0] * m[0, 1]) * invdet

    return minv



c_inv = inv_m(M.T@M)
np_inv = np.linalg.inv(M.T@M)

c_inv - np_inv


array([[ 0.00000000e+00,  4.81482486e-35,  0.00000000e+00],
       [ 4.81482486e-35,  0.00000000e+00, -1.08420217e-19],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [28]:
theta_c_1 = MtY1*c_inv[0,0] + MtY2*c_inv[0,1] + MtY3*c_inv[0,2]
theta_c_2 = MtY1*c_inv[1,0] + MtY2*c_inv[1,1] + MtY3*c_inv[1,2]
theta_c_3 = MtY1*c_inv[2,0] + MtY2*c_inv[2,1] + MtY3*c_inv[2,2]

print('C-code tehta', theta_c_1, theta_c_2, theta_c_3)
print("true theta", theta_true)

C-code tehta 0.13369999999999932 0.4200000000000008 4.336999999999998
true theta [0.1337 0.42   4.337 ]
